In [1]:
import os
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv

In [2]:
load_dotenv()

# config and credentials
server = os.getenv('server')
database = os.getenv('database')
username = os.getenv('username')
password = os.getenv('password')

# connection
engine = db.create_engine(
    'mssql://{}:{}@{}/{}?driver=ODBC+Driver+18+for+SQL+Server'.format(
        username, password, server, database
    )
)

# establish connection
connection = engine.connect()

### Daily Distinct Cusips and Total Volume by Retail / Institunional Grouping

In [3]:
def distinctCusips_totalVolume_byRetailInstitutional(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT 
            TrdExctnDt,
            RetailThreshold,
            COUNT(DISTINCT CusipId) AS DistinctCusips,
            SUM(EntrdVolQt) AS TotalVolume
        FROM (
            SELECT
                TrdExctnDt,
                CASE WHEN EntrdVolQt < 250000 THEN 'R' WHEN EntrdVolQt >= 500000 THEN 'IN' END AS RetailThreshold,
                CusipId,
                EntrdVolQt
            FROM
                TraceFilteredWithRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        ) A
        GROUP BY
            TrdExctnDt,
            RetailThreshold
        ORDER BY
            TrdExctnDt, 
            RetailThreshold
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [4]:
start, end = 2002, 2022

df = distinctCusips_totalVolume_byRetailInstitutional(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/distinctCusips_totalVolume_byRetailInstitutional_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,RetailThreshold,DistinctCusips,TotalVolume
0,2002-07-01,None,266,1.595870e+08
1,2002-07-01,IN,690,7.094470e+09
2,2002-07-01,R,1490,2.709536e+08
3,2002-07-02,None,285,1.622870e+08
4,2002-07-02,IN,738,1.321157e+10


### Daily Distinct Cusips and Total Volume

In [5]:
def distinctCusips_totalVolume(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT 
            TrdExctnDt,
            COUNT(DISTINCT CusipId) AS DistinctCusips,
            SUM(EntrdVolQt) AS TotalVolume
        FROM
            TraceFilteredWithRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        GROUP BY
            TrdExctnDt
        ORDER BY
            TrdExctnDt
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [6]:
start, end = 2002, 2022

df = distinctCusips_totalVolume(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/distinctCusips_totalVolume_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,DistinctCusips,TotalVolume
0,2002-07-01,1840,7.525011e+09
1,2002-07-02,1875,1.371374e+10
2,2002-07-03,1533,4.861909e+09
3,2002-07-05,980,2.110523e+09
4,2002-07-08,1987,1.393342e+10


### Daily Total Volume by Side (Buy/Sell) 

In [7]:
def totalVolume_bySide(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT 
            TrdExctnDt,
            RptSideCd,
            SUM(EntrdVolQt) AS TotalVolume,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM
            TraceFilteredWithRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
            
    base_query += '''
        GROUP BY
            TrdExctnDt, 
            RptSideCd
        ORDER BY
            TrdExctnDt, 
            RptSideCd
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [8]:
start, end = 2002, 2022

df = totalVolume_bySide(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/totalVolume_bySide_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,RptSideCd,TotalVolume,DistinctCusips
0,2002-07-01,B,3.311436e+09,1378
1,2002-07-01,S,4.213575e+09,1429
2,2002-07-02,B,7.042704e+09,1432
3,2002-07-02,S,6.671035e+09,1537
4,2002-07-03,B,2.082034e+09,1056


### Industry Analysis on Cusips and Issuers

In [9]:
def distinctCusips_distinctIssuers_byIndustry(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            IndustryCode,
            DistinctCusips,
            DistinctIssuers
        FROM (
            SELECT 
                IndustryCode,
                COUNT(DISTINCT CusipId) AS DistinctCusips,
                COUNT(DISTINCT IssuerID) AS DistinctIssuers
            FROM
                TraceFilteredWithRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
            GROUP BY
                IndustryCode 
        ) A
        ORDER BY
            IndustryCode
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [10]:
start, end = 2002, 2022

df = distinctCusips_distinctIssuers_byIndustry(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/distinctCusips_distinctIssuers_byIndustry_{}-{}.csv'.format(start, end), index=False)
df.head()

,IndustryCode,DistinctCusips,DistinctIssuers
0,10,5778,851
1,11,1448,247
2,12,1596,294
3,13,82,11
4,14,1089,146


### Daily Distinct Cusips and Total Volume by Issuer

In [11]:
def distinctCusips_totalVolume_byIssuer(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            TrdExctnDt,
            IssuerId,
            COUNT(DISTINCT CusipId) AS DistinctCusips,
            SUM(EntrdVolQt) AS TotalVolume
        FROM 
            TraceFilteredWithRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        GROUP BY
            TrdExctnDt, 
            IssuerId
        ORDER BY
            TrdExctnDt, 
            IssuerId
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [12]:
start, end = 2002, 2022

df = distinctCusips_totalVolume_byIssuer(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/distinctCusips_totalVolume_byIssuer_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,IssuerId,DistinctCusips,TotalVolume
0,2002-07-01,11,5,7968000.0
1,2002-07-01,20,1,5000.0
2,2002-07-01,26,13,38126000.0
3,2002-07-01,34,1,200000.0
4,2002-07-01,41,3,9613000.0


### Daily Distinct Cusips by Rating

In [13]:
def distinctCusips_byRating(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            TrdExctnDt,
            RatingNum,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM 
            TraceFilteredWithRatings A
        WHERE
            A.TrdExctnDt >= '{}-01-1' AND A.TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        GROUP BY
            TrdExctnDt,
            RatingNum
        ORDER BY
            TrdExctnDt,
            RatingNum
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [14]:
start, end = 2002, 2022

df = distinctCusips_byRating(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/distinctCusips_byRating_{}-{}.csv'.format(start, end), index=False)
df.head()

,TrdExctnDt,RatingNum,DistinctCusips
0,2002-07-01,1.0,58
1,2002-07-01,2.0,68
2,2002-07-01,3.0,112
3,2002-07-01,4.0,182
4,2002-07-01,5.0,186


### Distinct Cusips per Maturity Band

In [15]:
def distinctCusips_byMaturity(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            MaturityBand,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM (
            SELECT
                CASE 
                    WHEN ABS(DATEDIFF(DAY, Maturity, OfferingDate)) * 1.0 / 360 < 5 THEN 1
                    WHEN ABS(DATEDIFF(DAY, Maturity, OfferingDate)) * 1.0 / 360 < 15 THEN 2
                    ELSE 3
                END AS MaturityBand,
                CusipId
            FROM
                TraceFilteredWithRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    base_query += '''
        ) A
        GROUP BY
            MaturityBand
        ORDER BY
            MaturityBand
    '''
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [16]:
start, end = 2002, 2022

df = distinctCusips_byMaturity(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/distinctCusips_byMaturity_{}-{}.csv'.format(start, end), index=False)
df.head()

,MaturityBand,DistinctCusips
0,1,4602
1,2,17477
2,3,4508


### Distinct Cusips per Investment Grade

In [17]:
def distinctCusips_byInvestmentGrade(year_start, year_end):
    
    base_query = '''
        SELECT
            InvestmentGrade,
            COUNT(DISTINCT CusipId) AS DistinctCusips
        FROM (
            SELECT
                CusipId,
                CASE
                    WHEN RatingNum = 0 THEN 'NR'
                    WHEN RatingNum < 11 THEN 'IG'
                    ELSE 'HY'
                END AS InvestmentGrade
            FROM 
                TraceFilteredWithRatings
            WHERE
                TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
        ) A
        GROUP BY
            InvestmentGrade
        ORDER BY
            InvestmentGrade
    '''.format(
        year_start,
        year_end + 1
    )
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [18]:
start, end = 2002, 2022

df = distinctCusips_byInvestmentGrade(year_start=start, year_end=end)
df.to_csv('source/distinctCusips_byInvestmentGrade_{}-{}.csv'.format(start, end), index=False)
df.head()

,InvestmentGrade,DistinctCusips
0,HY,5961
1,IG,23049
2,NR,45019


### Distinct Trading Days

In [19]:
def distinctTradingDays(year_start, year_end, discard_nr):
    
    base_query = '''
        SELECT
            COUNT(DISTINCT TrdExctnDt)
        FROM
            TraceFilteredWithRatings
        WHERE
            TrdExctnDt >= '{}-01-1' AND TrdExctnDt < '{}-01-01'
    '''.format(
        year_start,
        year_end + 1
    )
    
    if discard_nr:
        base_query += 'AND RatingNum <> 0'
    
    df = pd.read_sql(base_query, connection)
    
    return df

In [20]:
start, end = 2002, 2022 # MIN = 2002-07-01 | MAX = 2022-09-30

df = distinctTradingDays(year_start=start, year_end=end, discard_nr=True)
df.to_csv('source/distinctTradingDays_{}-{}.csv'.format(start, end), index=False)
df.head()

,
0,5099
